<a href="https://colab.research.google.com/github/EduGalvan/XGBoost/blob/main/XGBOOST_salarios.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/adult.data', sep = ', ', header= None)

/usr/local/lib/python3.8/dist-packages/pandas/util/_decorators.py:311: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return func(*args, **kwargs)


In [ ]:
columnas = ['age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status', 'occupation', 'relationship','race', 'sex', 'capital-gain', 'capital-loss', 'hours-per-week', 'native-country', 'target' ]

In [ ]:
df.columns = columnas

In [ ]:
df

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,target,40 semanales,ganancias netas
0,39,1298,77516,5355,13,10683,3770,8305,27816,0,2174,0,40,29170,0,0,2174
1,50,2541,83311,5355,13,14976,4066,13193,27816,0,0,0,13,29170,0,0,0
2,38,22696,215646,10501,9,4443,1370,8305,27816,0,0,0,40,29170,0,0,0
3,53,22696,234721,1175,7,14976,1370,13193,3124,0,0,0,40,29170,0,0,0
4,28,22696,338409,5355,13,14976,4140,1568,3124,1,0,0,40,95,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,27,22696,257302,1067,12,14976,928,1568,27816,1,0,0,38,29170,0,0,0
32557,40,22696,154374,10501,9,14976,2002,13193,27816,0,0,0,40,29170,1,0,0
32558,58,22696,151910,10501,9,993,3770,3446,27816,1,0,0,40,29170,0,0,0
32559,22,22696,201490,10501,9,10683,3770,5068,27816,0,0,0,20,29170,0,0,0


In [ ]:
df['target'] = np.where(df['target'] == '<=50K', 0, 1)

In [ ]:
df['40 semanales'] = np.where(df['hours-per-week'] <= 40, 0, 1)

In [ ]:
df['ganancias netas'] = df['capital-gain'] - df['capital-loss']

In [ ]:
df['sex'] = np.where(df['sex'] == 'Female', 1, 0)

In [ ]:
#trnaformar las clases categroicas a numericas sustituyendo la categoria por su frecuencia
a = df.groupby('workclass')['workclass'].transform('count')


In [ ]:
df.columns


Index(['age', 'workclass', 'fnlwgt', 'education', 'education-num',
       'marital-status', 'occupation', 'relationship', 'race', 'sex',
       'capital-gain', 'capital-loss', 'hours-per-week', 'native-country',
       'target', '40 semanales', 'ganancias netas'],
      dtype='object')

In [ ]:
cat_col = ['workclass', 'education','marital-status', 'occupation', 'relationship', 'race', 'native-country']

In [ ]:
for i in cat_col:
  df[i] = df.groupby(i)[i].transform('count') 


In [ ]:
X = df.loc[:, df.columns!='target']

In [ ]:
y = df.loc[:,'target']

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 0.5, random_state= 1)

In [ ]:
X_valid, X_test, y_valid, y_test = train_test_split(X_test, y_test, test_size= 0.5, random_state= 1)

In [ ]:
X_train.shape, X_test.shape, X_valid.shape

((16280, 16), (8141, 16), (8140, 16))

In [ ]:
import xgboost

In [ ]:
xgb = xgboost.XGBClassifier()

In [ ]:
parameters = {'nthreads': [1],
              'objective': ['binary_logistic'],
              'learning_rate': [0.01, 0.05],
              'n_estimators': [100, 200],
              'early_stoping_rounds': [10],
              'eval_metric': ['logloss'],
              'eval_set': [(X_test, y_test)]
              }

In [ ]:
#se importa un algortimo de cross validation para encontrar la mejor combinacion de parametros
from sklearn.model_selection import GridSearchCV

In [ ]:
fit_params = {'early_stoping_rounds': 10,
              'eval_metric': 'logloss',
              'eval_set': [(X_test, y_test)]}

In [ ]:
clf = GridSearchCV(xgb, parameters,
                   cv = 3, scoring= 'accuracy')

In [ ]:
clf.fit(X_train, y_train)

GridSearchCV(cv=3, estimator=XGBClassifier(),
             param_grid={'early_stoping_rounds': [10],
                         'eval_metric': ['logloss'],
                         'eval_set': [(       age  workclass  fnlwgt  education  education-num  marital-status  \
6074    55      22696  169611       1175              7             993   
17005   51      22696  120914        933              6           10683   
2232    45      22696  339506      10501              9           10683   
30410   64      22696  137205       1382             11           14976   
17996   30      22696  113364       5355             13           10683   
...    ...        ...     ...        ...            ...             ...   
12951   19      22696  3...
30410              48           29170             1                0  
17996              18           29170             0                0  
...               ...             ...           ...              ...  
12951              25           29170    

In [ ]:
clf.best_estimator_

In [ ]:
clf.best_score_

0.8622850489784404

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
bestxgb = clf.best_estimator_

In [ ]:
ypreds = bestxgb.predict(X_valid)

In [ ]:
comp = pd.DataFrame({'real': y_valid, 'preds': ypreds})

In [ ]:
comp

,real,preds
22206,0,0
15112,0,0
4426,0,0
4960,0,0
32157,0,0
...,...,...
5080,0,0
10471,0,0
13413,0,0
32207,0,0


In [ ]:
acc = accuracy_score(y_valid, ypreds)

In [ ]:
acc

0.8730958230958231